# WeRateDogs Twitter Analysis

This project was done as a Udacity project to put together the concepts we learned. The Twitter handle [WeRateDogs](twitter.com/dog_rates). The key thing this channel does that this project was focused on is extracting the ratings, most of which are 12/10 or some other rating over 10.

As a disclaimer, Udacity is absolute garbage and I would not recomend it to anyone as there is no support. 

## Step 1 - Gathering the Data
There are three data sources and their details are listed below:
* `img_predictions`: This is a file that was gather from Udacity using requests package. The contents include tweet_data with AI predictions as to what the dogs are in the images. 
* `post_archive`: This is a file that was provided from Udacity for download to local machine and included 5,000 posts that has been modified to pull out the dog ratings from the text strings of the posts. Udacity intentionally left key metrics out of this document.
* `tweet_data`: This data source pulls from Twitter's API directly using the tweet_id variable from post_archive. 

The code block below imports our python modules as well as our secrete API credentials so that this notebook can utilize a public git repo.

In [98]:
import requests
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import os.path
import API_Key_Credentials as creds

consumer_key = creds.consumer_key
consumer_secret = creds.consumer_secret
access_token = creds.access_token
access_secret = creds.access_secret

**Gathering** `img_predictions`

In [99]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [100]:
response.status_code

200

In [101]:
if response.status_code == 200:
    print("We're in boys")
else:
    print("Houston, we have a problem, Error: ",response.status_code)

We're in boys


In [102]:
with open('image_predictions.tsv', 'wb') as file:
    file.write(response.content)

In [103]:
img_predictions = pd.read_csv('image_predictions.tsv', delimiter='\t')
img_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1741,822610361945911296,https://pbs.twimg.com/media/C2p_wQyXEAELtvS.jpg,1,cocker_spaniel,0.664487,True,Norfolk_terrier,0.075089,True,Norwich_terrier,0.059644,True
1336,758355060040593408,https://pbs.twimg.com/media/CoY324eWYAEiDOG.jpg,1,Pembroke,0.987643,True,Cardigan,0.012112,True,Siamese_cat,0.000117,False
1468,778990705243029504,https://pbs.twimg.com/media/Cs-H5uhWcAAiNY9.jpg,2,cocker_spaniel,0.715351,True,Labrador_retriever,0.207056,True,Chihuahua,0.028519,True
814,692568918515392513,https://pbs.twimg.com/media/CZx_wV2UMAArgsJ.jpg,2,golden_retriever,0.636845,True,Labrador_retriever,0.163362,True,Pekinese,0.045554,True
727,686377065986265092,https://pbs.twimg.com/media/CYaAS2kUoAINkye.jpg,1,German_shepherd,0.830816,True,Leonberg,0.076325,True,bloodhound,0.037449,True


**Gathering** `post_archive`

In [104]:
post_archive = pd.read_csv('twitter-archive-enhanced.csv')
post_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1705,680836378243002368,NaN,NaN,2015-12-26 19:43:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ellie. She's secretly ferocious. 12/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680836378...,12,10,Ellie,None,None,pupper,None
273,840728873075638272,NaN,NaN,2017-03-12 00:59:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Pipsy. He is a fluffbal...,6.671522e+17,4.196984e+09,2015-11-19 01:27:25 +0000,https://twitter.com/dog_rates/status/667152164...,12,10,Pipsy,None,None,None,None
1104,735137028879360001,NaN,NaN,2016-05-24 15:55:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Buckley. His family &amp; some neighbors ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/735137028...,9,10,Buckley,None,None,pupper,None
2132,670079681849372674,NaN,NaN,2015-11-27 03:20:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Fabio. He's a wonderful pup. Can't stay a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670079681...,10,10,Fabio,None,None,None,None
2178,669015743032369152,NaN,NaN,2015-11-24 04:52:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Super rare dog right here guys. Doesn't bark. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669015743...,10,10,None,None,None,None,None


**Gathering** `tweet_data`

In [105]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

tweet_ids = post_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()


# Save each tweet's returned JSON as a new line in a .txt file
if os.path.exists('tweet_json.txt'):
    print('Twitter Info Already Captured')
else:
    with open('tweet_json.txt', 'w') as outfile:
        # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.TweepError as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
end = timer()


Twitter Info Already Captured


This codeblock below took a while to figure out but was my solution to storing the raw JSON from a txt file to python object. It stores each JSON as a dictionary inside the `tweet_data` list. Note that `for line in f:` is a special sytax that parses each line of the txt file.

In [106]:
with open('tweet_json.txt') as f:
    tweet_data = []
    for line in f:
        tweet_data.append(json.loads(line))
    

In [107]:
type(tweet_data)

list

In [108]:
type(tweet_data[1])

dict

Example of the raw API with all it's attributes before selectively choosing which ones to add to our  master dataFrame

In [109]:
tweet_data[4]

{'created_at': 'Sat Jul 29 16:00:24 +0000 2017',
 'id': 891327558926688256,
 'id_str': '891327558926688256',
 'full_text': 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f',
 'truncated': False,
 'display_text_range': [0, 138],
 'entities': {'hashtags': [{'text': 'BarkWeek', 'indices': [129, 138]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 891327551943041024,
    'id_str': '891327551943041024',
    'indices': [139, 162],
    'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'url': 'https://t.co/AtUZn91f7f',
    'display_url': 'pic.twitter.com/AtUZn91f7f',
    'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1',
    'type': 'photo',
    'sizes': {'medium': {'w': 720, 'h': 540, 'resize': 'fit'},
     'large': {'w': 720, 'h':

In [110]:
tweet_data[4]['retweet_count']

8837

## Step 2 - Creating Master dataFrame
This process is all about merging our three datasets then going through the process of accessing and cleaning.

This codeblock transforms our list of dictionaries to a dataFrame. Some did not contain media which would throw an error. Because in this instance we are only interested in posts with photos using a simple pass funtion solved our issue appropiately.

In [111]:
tweets = pd.DataFrame()

for i in range(1,len(tweet_data)):
    try:
        tweets = tweets.append({'tweet_id': tweet_data[i]['id'],
                                'retweets': tweet_data[i]['retweet_count'],
                                'favorite_counts': tweet_data[i]['favorite_count'],
                                'created_at': tweet_data[i]['created_at'],
                                'full_text':tweet_data[i]['full_text'].split('https://')[0],
                                'post_link':tweet_data[i]['full_text'].split('https://')[1],
                                'picture':tweet_data[i]['entities']['media'][0]['media_url'],
                                'retweeted':tweet_data[i]['retweeted']
                               }, ignore_index=True)
    except:
        pass

In [112]:
tweets.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id']

### Question:
Is it possible to merge three dataFrames at the same time in a pythonic fashion?

In [113]:
master_df = pd.merge(tweets, img_predictions, on='tweet_id')
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,Mon Jul 31 00:18:03 +0000 2017,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
1,Sun Jul 30 15:58:51 +0000 2017,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2,Sat Jul 29 16:00:24 +0000 2017,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
3,Sat Jul 29 00:08:17 +0000 2017,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
4,Fri Jul 28 00:22:40 +0000 2017,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
5,Thu Jul 27 16:25:51 +0000 2017,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
6,Wed Jul 26 00:31:25 +0000 2017,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True
7,Tue Jul 25 00:10:02 +0000 2017,26144.0,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,0.0,4251.0,8.896388e+17,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True
8,Mon Jul 24 00:19:32 +0000 2017,24343.0,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,0.0,5063.0,8.892788e+17,https://pbs.twimg.com/ext_tw_video_thumb/88927...,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True
9,Sun Jul 23 00:22:39 +0000 2017,28106.0,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,0.0,4246.0,8.889172e+17,https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True


In [114]:
master_df = pd.merge(master_df, post_archive, on='tweet_id')
master_df.sample(20)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
981,Sun Dec 13 18:10:33 +0000 2015,2844.0,Meet Striker. He's ready for Christmas. 11/10,http://pbs.twimg.com/media/CWH_FTgWIAAwOUy.jpg,t.co/B3xxSLjQSH,0.0,1179.0,6.761019e+17,https://pbs.twimg.com/media/CWH_FTgWIAAwOUy.jpg,1,Shih-Tzu,0.225848,True,Norfolk_terrier,0.186873,True,Irish_terrier,0.106987,True,NaN,NaN,2015-12-13 18:10:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Striker. He's ready for Christmas. 11/10 ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676101918...,11,10,Striker,None,None,None,None
498,Sat Jul 02 02:17:13 +0000 2016,4947.0,Meet Winston. He's pupset because I forgot to ...,http://pbs.twimg.com/media/CmU2DT8WIAAwOXx.jpg,t.co/xEY8dbJxnF,0.0,1575.0,7.490644e+17,https://pbs.twimg.com/media/CmU2DVWWgAArvp3.jpg,2,pug,0.985222,True,Brabancon_griffon,0.003314,True,Pekinese,0.002989,True,NaN,NaN,2016-07-02 02:17:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Winston. He's pupset because I forgot to ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749064354...,11,10,Winston,None,None,None,None
349,Tue Sep 27 02:53:48 +0000 2016,12666.0,Meet Hercules. He can have whatever he wants f...,http://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg,t.co/mH0IOyFdIG,0.0,3517.0,7.806013e+17,https://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg,1,Saint_Bernard,0.995143,True,Cardigan,0.003044,True,English_springer,0.001050,True,NaN,NaN,2016-09-27 02:53:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Hercules. He can have whatever he wants f...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/780601303...,12,10,Hercules,None,None,None,None
706,Mon Feb 29 17:56:32 +0000 2016,8216.0,Say hello to Bisquick. He's a Beneplop Cumbers...,http://pbs.twimg.com/media/CcZn6RWWIAAmOZG.jpg,t.co/5zX5XD3i6K,0.0,3655.0,7.043646e+17,https://pbs.twimg.com/media/CcZn6RWWIAAmOZG.jpg,1,Pembroke,0.980695,True,Cardigan,0.018504,True,Chihuahua,0.000215,True,NaN,NaN,2016-02-29 17:56:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Bisquick. He's a Beneplop Cumbers...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704364645...,12,10,Bisquick,None,None,None,None
88,Tue May 02 00:04:57 +0000 2017,89061.0,We only rate dogs. This is quite clearly a smo...,http://pbs.twimg.com/ext_tw_video_thumb/859196...,t.co/g2nSyGenG9,0.0,29693.0,8.591970e+17,https://pbs.twimg.com/ext_tw_video_thumb/85919...,1,Angora,0.224218,False,malamute,0.216163,True,Persian_cat,0.128383,False,NaN,NaN,2017-05-02 00:04:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We only rate dogs. This is quite clearly a smo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/859196978...,12,10,quite,None,None,None,None
430,Mon Aug 01 23:15:56 +0000 2016,4102.0,This is Fido. He can tell the weather. Not goo...,http://pbs.twimg.com/media/Coz12OLWgAADdys.jpg,t.co/4gOv2Q3iKP,0.0,904.0,7.602528e+17,https://pbs.twimg.com/media/Coz12OLWgAADdys.jpg,1,radio_telescope,0.155279,False,dam,0.154515,False,crane,0.098040,False,NaN,NaN,2016-08-01 23:15:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Fido. He can tell the weather. Not goo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/760252756...,4,10,Fido,None,None,None,None
1263,Sat Nov 21 03:44:27 +0000 2015,487.0,Wow. Armored dog here. Ready for battle. Face ...,http://pbs.twimg.com/media/CUTl5m1WUAAabZG.jpg,t.co/SyMoWrp368,0.0,300.0,6.679114e+17,https://pbs.twimg.com/media/CUTl5m1WUAAabZG.jpg,1,frilled_lizard,0.257695,False,ox,0.235160,False,triceratops,0.085317,False,NaN,NaN,2015-11-21 03:44:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Wow. Armored

## Step 3 - Cleaning the Master dataFrame

In [115]:
master_df.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [116]:
#Transforms the appropiate fields to datetime format
time_variables = ['created_at','timestamp','retweeted_status_timestamp']
for var in time_variables:
    master_df[var] = pd.to_datetime(master_df[var])

In [117]:
#Checking dTypes
master_df.dtypes

created_at                    datetime64[ns, UTC]
favorite_counts                           float64
full_text                                  object
picture                                    object
post_link                                  object
retweeted                                 float64
retweets                                  float64
tweet_id                                  float64
jpg_url                                    object
img_num                                     int64
p1                                         object
p1_conf                                   float64
p1_dog                                       bool
p2                                         object
p2_conf                                   float64
p2_dog                                       bool
p3                                         object
p3_conf                                   float64
p3_dog                                       bool
in_reply_to_status_id                     float64


In [118]:
#Searching for null values
master_df.isnull().sum()

created_at                       0
favorite_counts                  0
full_text                        0
picture                          0
post_link                        0
retweeted                        0
retweets                         0
tweet_id                         0
jpg_url                          0
img_num                          0
p1                               0
p1_conf                          0
p1_dog                           0
p2                               0
p2_conf                          0
p2_dog                           0
p3                               0
p3_conf                          0
p3_dog                           0
in_reply_to_status_id         1327
in_reply_to_user_id           1327
timestamp                        0
source                           0
text                             0
retweeted_status_id           1297
retweeted_status_user_id      1297
retweeted_status_timestamp    1297
expanded_urls                    0
rating_numerator    

In [119]:
pd.set_option('display.max_columns', 500)
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,2017-07-31 00:18:03+00:00,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
1,2017-07-30 15:58:51+00:00,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
2,2017-07-29 16:00:24+00:00,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
3,2017-07-29 00:08:17+00:00,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,NaN,NaN,2017-07-29 00:08:17+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
4,2017-07-28 00:22:40+00:00,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,NaN,NaN,2017-07-28 00:22:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
5,2017-07-27 16:25:51+00:00,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,NaN,NaN,2017-07-27 16:25:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
6,2017-07-26 00:31:25+00:00,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True,NaN,NaN,2017-07-26 00:31:25+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is K

In [120]:
master_df.drop(columns={'timestamp','jpg_url','source','text','expanded_urls','in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'}, inplace=True)

In [121]:
master_df.retweets = master_df.retweets.astype(int)
master_df.favorite_counts = master_df.favorite_counts.astype(int)
master_df.retweeted = master_df.retweeted.astype(int)
master_df.retweeted = master_df.retweeted.astype(bool)
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,2017-07-31 00:18:03+00:00,24183,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,False,3925,8.918152e+17,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,12,10,Archie,None,None,None,None
1,2017-07-30 15:58:51+00:00,40629,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,False,8161,8.916896e+17,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,13,10,Darla,None,None,None,None
2,2017-07-29 16:00:24+00:00,38862,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,False,8837,8.913276e+17,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,12,10,Franklin,None,None,None,None
3,2017-07-29 00:08:17+00:00,19532,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,False,2940,8.910880e+17,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,13,10,None,None,None,None,None
4,2017-07-28 00:22:40+00:00,62928,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,False,17831,8.907292e+17,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,13,10,None,None,None,None,None
5,2017-07-27 16:25:51+00:00,26866,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,False,4041,8.906092e+17,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,13,10,Zoey,None,None,None,None
6,2017-07-26 00:31:25+00:00,29605,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,False,6925,8.900066e+17,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True,13,10,Koda,None,None,None,None
7,2017-07-25 00:10:02+00:00,26144,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,False,4251,8.896388e+17,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True,12,10,Ted,None,None,None,None
8,2017-07-24 00:19:32+00:00,24343,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,False,5063,8.892788e+17,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True,13,10,Oliver,None,None,None,None
9,2017-07-23 00:22:39+00:00,28106,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,False,4246,8.889172e+17,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True,12,10,Jim,None,None,None,None


In [122]:
melt = pd.melt(master_df, id_vars=(master_df.columns.tolist()[0:-4]), value_name='dog_category')
melt

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,variable,dog_category
0,2017-07-31 00:18:03+00:00,24183,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,False,3925,8.918152e+17,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,12,10,Archie,doggo,None
1,2017-07-30 15:58:51+00:00,40629,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,False,8161,8.916896e+17,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,13,10,Darla,doggo,None
2,2017-07-29 16:00:24+00:00,38862,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,False,8837,8.913276e+17,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,12,10,Franklin,doggo,None
3,2017-07-29 00:08:17+00:00,19532,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,False,2940,8.910880e+17,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,13,10,None,doggo,None
4,2017-07-28 00:22:40+00:00,62928,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,False,17831,8.907292e+17,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,13,10,None,doggo,None
5,2017-07-27 16:25:51+00:00,26866,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,False,4041,8.906092e+17,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,13,10,Zoey,doggo,None
6,2017-07-26 00:31:25+00:00,29605,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,False,6925,8.900066e+17,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True,13,10,Koda,doggo,None
7,2017-07-25 00:10:02+00:00,26144,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,False,4251,8.896388e+17,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True,12,10,Ted,doggo,None
8,2017-07-24 00:19:32+00:00,24343,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,False,5063,8.892788e+17,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True,13,10,Oliver,doggo,None
9,2017-07-23 00:22:39+00:00,28106,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,False,4246,8.889172e+17,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True,12,10,Jim,doggo,None


In [123]:
melt.dog_category.unique()

array(['None', 'doggo', 'floofer', 'pupper', 'puppo'], dtype=object)

In [124]:
melt.query("dog_category in ['doggo', 'floofer', 'pupper', 'puppo']")

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,variable,dog_category
58,2017-06-09 00:02:31+00:00,26443,Here's a very large dog. He has a date later. ...,http://pbs.twimg.com/media/DB1m871XkAUbCkY.jpg,t.co/EMYIdoblMR,False,5158,8.729671e+17,2,Labrador_retriever,0.476913,True,Chesapeake_Bay_retriever,0.174145,True,German_short-haired_pointer,0.092861,True,12,10,None,doggo,doggo
89,2017-05-01 00:40:27+00:00,15403,I have stumbled puppon a doggo painting party....,http://pbs.twimg.com/media/C-s5oYZXkAAMHHq.jpg,t.co/cUeDMlHJbq,False,3438,8.588435e+17,1,golden_retriever,0.578120,True,Labrador_retriever,0.286059,True,bloodhound,0.026917,True,13,10,None,doggo,doggo
99,2017-04-22 18:31:02+00:00,45445,Here's a puppo participating in the #ScienceMa...,http://pbs.twimg.com/media/C-CYWrvWAAU8AXH.jpg,t.co/cMhq16isel,False,17833,8.558515e+17,1,flat-coated_retriever,0.321676,True,Labrador_retriever,0.115138,True,groenendael,0.096100,True,13,10,None,doggo,doggo
104,2017-04-17 16:34:26+00:00,16239,"At first I thought this was a shy doggo, but i...",http://pbs.twimg.com/media/C9oNt91WAAAFSLS.jpg,t.co/TXdT3tmuYk,False,3168,8.540102e+17,1,English_springer,0.354733,True,collie,0.177538,True,Border_collie,0.131706,True,11,10,None,doggo,doggo
124,2017-03-24 22:08:59+00:00,7792,Say hello to Mimosa. She's an emotional suppor...,http://pbs.twimg.com/media/C7t0IzLWkAINoft.jpg,t.co/L6mLzrd7Mx,False,1850,8.453971e+17,1,Dandie_Dinmont,0.394404,True,Maltese_dog,0.186537,True,West_Highland_white_terrier,0.181985,True,13,10,Mimosa,doggo,doggo
153,2017-02-16 17:00:25+00:00,11690,Say hello to Smiley. He's a blind therapy dogg...,http://pbs.twimg.com/ext_tw_video_thumb/832273...,t.co/SHAb1wHjMz,False,2431,8.322734e+17,1,Pembroke,0.134081,True,ice_bear,0.051928,False,pug,0.044311,True,14,10,Smiley,doggo,doggo
163,2017-02-08 22:00:52+00:00,10835,Here's a stressed doggo. Had a long day. Many ...,http://pbs.twimg.com/media/C4LMUf8WYAkWz4I.jpg,t.co/fmRS43mWQB,False,2113,8.294499e+17,1,Labrador_retriever,0.315163,True,golden_retriever,0.153210,True,Pekinese,0.132791,True,11,10,None,doggo,doggo
171,2017-02-01 17:44:55+00:00,37787,This is Cupid. He was found in the trash. Now ...,http://pbs.twimg.com/media/C3mOnZ8WMAAQXRY.jpg,t.co/WS0Gha8vRh,False,10680,8.268488e+17,4,Great_Pyrenees,0.858764,True,golden_retriever,0.023526,True,Pekinese,0.017104,True,13,10,Cupid,doggo,doggo
189,2017-01-18 17:07:18+00:00,8764,This is Duchess. She uses dark doggo forces to...,http://pbs.twimg.com/media/C2d_vnHWEAE9phX.jpg,t.co/maDNMETA52,False,1737,8.217659e+17,1,golden_retriever,0.980071,True,Labrador_retriever,0.008758,True,Saluki,0.001806,True,13,10,Duchess,doggo,doggo
194,2017-01-13 15:08:56+00:00,13447,Here we have a doggo who has messed up. He was...,http://pbs.twimg.com/ext_tw_video_thumb/819924...,t.co/XdRNXNYD4E,False,5125,8.199242e+17,1,bathtub,0.100896,False,shower_curtain,0.091866,False,tub,0.049176,False,11,10,None,doggo,doggo


In [125]:
melt.dog_category = melt.variable

In [126]:
melt.drop(columns='variable', inplace=True)

In [128]:
melt.sample(50)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,dog_category
1345,2017-07-29 00:08:17+00:00,19532,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,False,2940,8.910880e+17,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,13,10,None,floofer
3700,2015-12-10 04:18:42+00:00,862,When your entire life is crumbling before you ...,http://pbs.twimg.com/ext_tw_video_thumb/674805...,t.co/vqFkgYPCW8,False,358,6.748054e+17,1,English_springer,0.594467,True,cocker_spaniel,0.389994,True,Welsh_springer_spaniel,0.007096,True,10,10,None,pupper
2392,2015-12-08 00:37:11+00:00,1787,When you realize it doesn't matter how hard yo...,http://pbs.twimg.com/media/CVqeEKLW4AA1wXH.jpg,t.co/qzYXbyv0SJ,False,1219,6.740249e+17,1,Pomeranian,0.648500,True,Pekinese,0.339835,True,Persian_cat,0.006448,False,10,10,None,floofer
2090,2016-02-14 21:55:47+00:00,3461,This is Oliver. He does toe touches in his sle...,http://pbs.twimg.com/media/CbNO0DaW0AARcki.jpg,t.co/3BrRIWjG35,False,1008,6.989890e+17,1,Norfolk_terrier,0.246340,True,Irish_terrier,0.243349,True,golden_retriever,0.085871,True,13,10,Oliver,floofer
2593,2015-11-22 03:46:11+00:00,814,Meet Sophie. Her son just got in the car to le...,http://pbs.twimg.com/media/CUYv4d2WUAAziXs.jpg,t.co/3j9kZRcpVB,False,228,6.682742e+17,1,soft-coated_wheaten_terrier,0.406374,True,Lakeland_terrier,0.263854,True,toy_poodle,0.150844,True,10,10,Sophie,floofer
2761,2017-05-12 16:05:02+00:00,8349,Say hello to Quinn. She's quite the goofball. ...,http://pbs.twimg.com/media/C_o2vKFV0AAcq9j.jpg,t.co/MOBkQnyHib,False,2378,8.630625e+17,2,French_bulldog,0.935804,True,pug,0.059576,True,boxer,0.001412,True,13,10,Quinn,pupper
1493,2017-02-17 17:01:29+00:00,16361,This is Orion. He just got back from the denti...,http://pbs.twimg.com/media/C44eG7oUMAAA4Ss.jpg,t.co/Y4DZx2UWsr,False,2897,8.326361e+17,1,Eskimo_dog,0.525032,True,Siberian_husky,0.252238,True,malamute,0.216839,True,12,10,Orion,floofer
1662,2016-10-16 18:11:26+00:00,10741,This is Tonks. She is a service puppo. Can hea...,http://pbs.twimg.com/media/Cu6I9vaWcAI6lYI.jpg,t.co/i622ZbWkUp,False,2947,7.877176e+17,3,German_shepherd,0.992339,True,malinois,0.004920,True,kelpie,0.000853,True,13,10,Tonks,floofer
3995,2015-11-18 03:18:55+00:00,505,This is Jeph. He is a German Boston Shuttlecoc...,http://pbs.twimg.com/media/CUEDSMEWEAAuXVZ.jpg,t.co/8whlkYw3mO,False,243,6.668178e+17,1,miniature_schnauzer,0.496953,True,standard_schnauzer,0.285276,True,giant_schnauzer,0.073764,True,9,10,Jeph,pupper
364,2016-09-19 01:42:24+00:00,11729,"""Yep... just as I suspected. You're not flossi...",http://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg,t.co/SuXcI9B7pQ,False,3093,7.776842e+17,1,cocker_spaniel,0.253442,True,golden_retriever,0.162850,True,otterhound,0.110921,True,12,10,None,doggo


In [143]:
#Tidy the names
p_columns = ['p1','p2','p3']
for pvar in p_columns:
    melt[pvar] = melt[pvar].str.replace('-','_')
    melt[pvar] = melt[pvar].str.lower()

melt.sample(150)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,dog_category
3884,2015-11-25 19:25:57+00:00,512,This is Skye. He is a Bretwaldian Altostratus....,http://pbs.twimg.com/media/CUrjvxiVEAA94dH.jpg,t.co/XmCvma01fF,False,151,6.695979e+17,1,eskimo_dog,0.595665,True,siberian_husky,0.214474,True,white_wolf,0.147235,False,10,10,Skye,pupper
4784,2016-02-11 00:18:49+00:00,2098,This is Ole. He's not sure how to gravity. 8/10,http://pbs.twimg.com/media/Ca5JMvMUsAAGMll.jpg,t.co/PsqqotpBBQ,False,513,6.975755e+17,1,siamese_cat,0.256698,False,whippet,0.119805,True,bull_mastiff,0.102595,True,8,10,Ole,puppo
2983,2016-10-31 17:00:11+00:00,6796,Your favorite squad is looking extra h*ckin sp...,http://pbs.twimg.com/media/CwHIg61WIAApnEV.jpg,t.co/PrgvOyPtDT,False,2638,7.931355e+17,1,bakery,0.737041,False,saltshaker,0.052396,False,teddy,0.046593,False,13,10,None,pupper
842,2016-01-13 16:56:30+00:00,20677,This is Tyrone. He's a leaf wizard. Self-motiv...,http://pbs.twimg.com/media/CYnXcLEUkAAIQOM.jpg,t.co/pRp1Npucbz,False,9448,6.873173e+17,1,shih_tzu,0.747208,True,maltese_dog,0.091025,True,lhasa,0.035788,True,11,10,Tyrone,doggo
1351,2017-07-23 00:22:39+00:00,28106,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,False,4246,8.889172e+17,1,golden_retriever,0.714719,True,tibetan_mastiff,0.120184,True,labrador_retriever,0.105506,True,12,10,Jim,floofer
2177,2016-01-16 04:11:31+00:00,2292,This is Derby. He's a superstar. 13/10 (vid by...,http://pbs.twimg.com/ext_tw_video_thumb/688211...,t.co/o4Nfc8WoAO,False,856,6.882120e+17,1,bannister,0.369449,False,four_poster,0.105307,False,shoji,0.098767,False,13,10,Derby,floofer
1250,2015-11-22 04:33:59+00:00,500,Meet Rusty. Rusty's dreaming of a world where ...,http://pbs.twimg.com/media/CUY60usWoAAdBxx.jpg,t.co/C8U6cxI1Jc,False,135,6.682863e+17,1,golden_retriever,0.215944,True,basset,0.189214,True,cardigan,0.113010,True,11,10,Rusty,doggo
1820,2016-07-08 16:44:23+00:00,3308,Here's a pupper that's very hungry but too laz...,http://pbs.twimg.com/ext_tw_video_thumb/751456...,t.co/lsVAMBq6ex,False,1043,7.514569e+17,1,golden_retriever,0.714409,True,afghan_hound,0.066163,True,chow,0.028413,True,12,10,None,floofer
1281,2015-11-20 02:32:25+00:00,458,Meet Naphaniel. He doesn't necessarily enjoy h...,http://pbs.twimg.com/media/CUOL0uGUkAAx7yh.jpg,t.co/xoRWyQTcmy,False,234,6.675309e+17,1,golden_retriever,0.633037,True,kuvasz,0.146391,True,labrador_retriever,0.046184,True,10,10,Naphaniel,doggo
3126,2016-07-28 01:00:57+00:00,4974,Why does this never happen at my front door......,http://pbs.twimg.com/ext_tw_video_thumb/758467...,t.co/HmwrdfEfUE,False,2315,7.584672e+17,1,labrador_retriever,0.436377,True,chihuahua,0.113956,True,american_staffordshire_terrier,0.099689,True,165,150,None,pupper


In [144]:
master_df = melt

In [149]:
master_df.drop_duplicates(subset='tweet_id', inplace=True)
master_df.sort_values('favorite_counts', ascending=False)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,dog_category
246,2016-12-09 06:17:20+00:00,124553,This is Stephan. He just wants to help. 13/10 ...,http://pbs.twimg.com/ext_tw_video_thumb/807106...,t.co/DkBYaCAg2d,False,58901,8.071068e+17,1,chihuahua,0.505370,True,pomeranian,0.120358,True,toy_terrier,0.077008,True,13,10,Stephan,doggo
198,2017-01-11 02:15:36+00:00,90604,This is Bo. He was a very good First Doggo. 14...,http://pbs.twimg.com/media/C12whDoVEAALRxa.jpg,t.co/AdPKrI8BZ1,False,38761,8.190048e+17,1,standard_poodle,0.351308,True,toy_poodle,0.271929,True,tibetan_terrier,0.094759,True,14,10,Bo,doggo
88,2017-05-02 00:04:57+00:00,89061,We only rate dogs. This is quite clearly a smo...,http://pbs.twimg.com/ext_tw_video_thumb/859196...,t.co/g2nSyGenG9,False,29693,8.591970e+17,1,angora,0.224218,False,malamute,0.216163,True,persian_cat,0.128383,False,12,10,quite,doggo
944,2015-12-20 02:20:55+00:00,80817,This made my day. 12/10 please enjoy,http://pbs.twimg.com/ext_tw_video_thumb/678399...,t.co/VRTbo3aAcm,False,32605,6.783997e+17,1,swing,0.929196,False,bedlington_terrier,0.015047,True,great_pyrenees,0.014039,True,12,10,None,doggo
63,2017-06-01 20:18:38+00:00,80570,This is Zoey. She really likes the planet. Wou...,http://pbs.twimg.com/media/DBQwlFCXkAACSkI.jpg,t.co/T1xlgaPujm,False,25363,8.703740e+17,1,golden_retriever,0.841001,True,great_pyrenees,0.099278,True,labrador_retriever,0.032621,True,13,10,Zoey,doggo
43,2017-06-22 03:54:17+00:00,76918,This is Aja. She was just told she's a good do...,http://pbs.twimg.com/media/DC5YqoPXgAA7Uph.jpg,t.co/lsPyyAiF1r,False,18048,8.777365e+17,2,chesapeake_bay_retriever,0.837956,True,labrador_retriever,0.062034,True,weimaraner,0.040599,True,13,10,Aja,doggo
979,2015-12-14 01:58:31+00:00,76421,This is Kenneth. He's stuck in a bubble. 10/10...,http://pbs.twimg.com/media/CWJqN9iWwAAg86R.jpg,t.co/uQt37xlYMJ,False,31481,6.762197e+17,1,bubble,0.997556,False,leafhopper,0.000159,False,whippet,0.000132,True,10,10,Kenneth,doggo
80,2017-05-10 00:08:34+00:00,73365,We only rate dogs. Please don't send perfectly...,http://pbs.twimg.com/media/C_bIo8MXkAA3xBK.jpg,t.co/nvZyyrp0kd,False,22694,8.620970e+17,2,chow,0.677589,True,pomeranian,0.270648,True,pekinese,0.038110,True,13,10,None,doggo
159,2017-02-12 01:04:29+00:00,69209,This is Lilly. She just parallel barked. Kindl...,http://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg,t.co/SATN4If5H5,False,17521,8.305833e+17,1,labrador_retriever,0.908703,True,seat_belt,0.057091,False,pug,0.011933,True,13,10,Lilly,doggo
4,2017-07-28 00:22:40+00:00,62928,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,False,17831,8.907292e+17,2,pomeranian,0.566142,True,eskimo_dog,0.178406,True,pembroke,0.076507,True,13,10,None,doggo


In [150]:
master_df.query("p1_conf > .9")

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,dog_category
6,2017-07-26 00:31:25+00:00,29605,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,False,6925,8.900066e+17,1,samoyed,0.957979,True,pomeranian,1.388350e-02,True,chow,8.167480e-03,True,13,10,Koda,doggo
7,2017-07-25 00:10:02+00:00,26144,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,False,4251,8.896388e+17,1,french_bulldog,0.991650,True,boxer,2.128640e-03,True,staffordshire_bullterrier,1.498180e-03,True,12,10,Ted,doggo
11,2017-07-20 16:49:33+00:00,20996,This is Gerald. He was just told he didn't get...,http://pbs.twimg.com/media/DFMWn56WsAAkA7B.jpg,t.co/DK7iDPfuRX,False,3283,8.880784e+17,1,french_bulldog,0.995026,True,pug,9.319080e-04,True,bull_mastiff,9.032110e-04,True,12,10,Gerald,doggo
19,2017-07-15 23:25:31+00:00,20417,This is Roscoe. Another pupper fallen victim t...,http://pbs.twimg.com/media/DE0BTnQUwAApKEH.jpg,t.co/RGE08MIJox,False,3012,8.863661e+17,1,french_bulldog,0.999201,True,chihuahua,3.611780e-04,True,boston_bull,7.556160e-05,True,12,10,Roscoe,doggo
20,2017-07-14 22:10:11+00:00,31459,Viewer discretion advised. This is Jimbo. He w...,http://pbs.twimg.com/media/DEumeWWV0AA-Z61.jpg,t.co/BuveP0uMF1,False,6383,8.859848e+17,1,blenheim_spaniel,0.972494,True,shih_tzu,6.630120e-03,True,bernese_mountain_dog,6.239150e-03,True,12,10,Jimbo,doggo
22,2017-07-13 01:35:06+00:00,0,RT @dog_rates: This is Lilly. She just paralle...,http://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg,t.co/SATN4If5H5,False,17521,8.853116e+17,1,labrador_retriever,0.908703,True,seat_belt,5.709090e-02,False,pug,1.193350e-02,True,13,10,Lilly,doggo
25,2017-07-10 15:58:53+00:00,26076,"I present to you, Pup in Hat. Pup in Hat is gr...",http://pbs.twimg.com/media/DEYrIZwWsAA2Wo5.jpg,t.co/vvBOcC2VdC,False,5372,8.844418e+17,1,pembroke,0.993225,True,cardigan,3.216480e-03,True,chihuahua,2.080890e-03,True,14,10,None,doggo
26,2017-07-08 00:28:19+00:00,44292,This is Bella. She hopes her smile made you sm...,http://pbs.twimg.com/media/DELC9dZXUAADqUk.jpg,t.co/qjrljjt948,False,9365,8.834828e+17,1,golden_retriever,0.943082,True,labrador_retriever,3.240900e-02,True,kuvasz,5.500720e-03,True,5,10,Bella,doggo
27,2017-07-07 00:17:54+00:00,35846,Please only send dogs. We don't rate mechanics...,http://pbs.twimg.com/media/DEF2-_jW0AEAS94.jpg,t.co/Se5fZ9wp5E,False,6296,8.831178e+17,2,golden_retriever,0.949562,True,labrador_retriever,4.594790e-02,True,kuvasz,2.470940e-03,True,13,10,None,doggo
35,2017-06-29 16:39:47+00:00,27522,This is Bella. She had her first beach experie...,http://pbs.twimg.com/media/DDgK-J4XUAIEV9W.jpg,t.co/4VsFysDmiw,False,5906,8.804658e+17,1,golden_retriever,0.913255,True,labrador_retriever,2.632860e-02,True,cocker_spaniel,9.370820e-03,True,12,10,Bella,doggo
